# Imports

In [ ]:
from datasets import load_dataset
from collections import Counter
import torch
import torch.nn as nn
from utils.datautils import *
from utils.MLutils import *
from transformers import BertTokenizerFast
from sklearn.model_selection import train_test_split
from transformers import BertModel

# linea que arregla algunos errores de loadeo de datasets
# pip install --upgrade datasets

# Procesamiento

In [ ]:
linux = True
device = None

if linux:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
else:
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

print("usando:", device)

# Ejericio b)

## Busqueda de fuentes

### Fuente 1: Conjunto de preguntas en espa;ol

In [ ]:
DATA_URLS = {
    "train": "https://huggingface.co/datasets/PlanTL-GOB-ES/SQAC/resolve/main/train.json",
    "dev":   "https://huggingface.co/datasets/PlanTL-GOB-ES/SQAC/resolve/main/dev.json",
    "test":  "https://huggingface.co/datasets/PlanTL-GOB-ES/SQAC/resolve/main/test.json",
}

raw = load_dataset(
    "json",
    data_files=DATA_URLS,
    field="data",
)

questions = []

for i in range(0, len(raw["train"])):
  for p in raw["train"][i]['paragraphs']:
    p_questions = [qas['question'] for qas in p['qas']]
    questions += p_questions

N_QUESTIONS = 5000  # Número de preguntas a procesar
questions = questions[:N_QUESTIONS] 

print(f"Se descargaron {len(questions)} preguntas en Español.")

### Fuente 2: Dataset provisto para Notebook 10

In [ ]:
dataset_rnn = load_dataset("google/wmt24pp", "en-es_MX", split="train")
oraciones_rnn = dataset_rnn['target'][1:]

print(f"Se descargaron {len(oraciones_rnn)} oraciones en Español (del dataset del notebook 10).")

### Fuente 3: Dataset sintetico generado con Gemini

In [ ]:
oraciones_sinteticas = []
import json
with open('./data/datasets.json', 'r') as file:
  data = json.load(file)

oraciones_sinteticas = data['otros'] + data['marcas']
print(f"Hay {len(oraciones_sinteticas)} oraciones sintéticas.")

### Fuente 4: Articulos de Wikipedia

In [ ]:
wikipedia.set_lang("es")

frases = obtener_frases_wikipedia("Revolución francesa", max_frases=50)
for f in frases[:5]:
    print(f"- {f}")

temas = [
    # Países y lugares
    'Argentina', 'España', 'México', 'Colombia', 'Chile',
    'Perú', 'Uruguay', 'Brasil', 'América Latina', 'Europa',

    # Cultura argentina
    'Lionel Messi', 'Diego Maradona', 'Lali Esposito', 'Charly Garcia', 'Dillom',
    'Tiempos Violentos', 'Relatos Salvajes', 'Universidad de Buenos Aires', 'Rock nacional', 'Cine argentino',

    # Historia y política
    'Revolucion de Mayo', 'Independencia de Argentina', 'Simón Bolívar', 'Segunda Guerra Mundial', 'Guerra Fría',
    'Revolución Francesa', 'Guerra Civil Española', 'Napoleón Bonaparte', 'Nelson Mandela', 'Dictadura militar en Argentina',

    # Ciencia y tecnología
    'Inteligencia artificial', 'ChatGPT', 'Redes neuronales', 'Robótica', 'Energía solar',
    'Vacunas', 'COVID-19', 'Cambio climático', 'Computadora cuántica', 'NASA',

    # Cultura general
    'El Principito', 'Premio Nobel', 'Frida Kahlo', 'Pablo Picasso', 'Leonardo da Vinci',
    'William Shakespeare', 'Gabriel García Márquez', 'Julio Cortázar', 'Literatura latinoamericana', 'Arte contemporáneo',

    # Entretenimiento y medios
    'Marvel', 'DC Comics', 'Netflix', 'Cine de terror', 'Películas de ciencia ficción',
    'Música electrónica', 'Reguetón', 'Spotify', 'YouTube', 'TikTok',

    # Deportes
    'Fútbol', 'Copa Mundial de la FIFA', 'Juegos Olimpicos', 'Tenis', 'NBA',
    'Boca Juniors', 'River Plate', 'Messi vs Ronaldo', 'Fórmula 1', 'Michael Jordan',

    # Sociedad y actualidad
    'Feminismo', 'Día Internacional de la Mujer', 'Diversidad cultural', 'Migración', 'Pobreza',
    'Educación pública', 'Salud mental', 'Medio ambiente', 'Derechos humanos', 'Trabajo remoto',

    # Filosofía y pensamiento
    'Filosofía', 'Ética', 'Psicología', 'Sigmund Freud', 'Carl Jung',
    'Existencialismo', 'Sociología', 'Economía', 'Política', 'Democracia'
]


# Para actualizar la info de Wikipedia, descomentar la siguiente linea
# cargar_json_wikipedia("frases_wikipedia.json",temas, max_frases=100)

# Guardar en un archivo JSON
with open("data/frases_wikipedia.json", "r", encoding="utf-8") as f:
    frases_wikipedia = json.load(f)

print(frases_wikipedia[:5])  # muestra las primeras frases

len(frases_wikipedia)

### Fuente 5: Subtitulos de peliculas

In [ ]:
esperando_la_carroza = extraer_frases_dialogo("data/esperando_la_carroza.txt")

with open("data/dialogos_esperando_la_carroza.json", "w", encoding="utf-8") as f:
    json.dump(esperando_la_carroza, f, ensure_ascii=False, indent=2)

print("✅ Frases extraídas y guardadas. Total:", len(esperando_la_carroza))
print(random.sample(esperando_la_carroza, 10))

In [ ]:
frases_relatos_salvajes = extraer_frases_subtitulos("data/subt_relatos_salvajes.srt")

# Guardar como JSON
with open("data/frases_relatos_salvajes.json", "w", encoding="utf-8") as f:
    json.dump(frases_relatos_salvajes, f, ensure_ascii=False, indent=2)

frases_relatos_salvajes = limpiar_simbolos(frases_relatos_salvajes)
esperando_la_carroza = limpiar_simbolos(esperando_la_carroza)

## Juntamos las fuentes

In [ ]:
oraciones_raw = questions + oraciones_rnn + oraciones_sinteticas + frases_wikipedia + esperando_la_carroza  + frases_relatos_salvajes

print('Cantidad total de oraciones:',len(oraciones_raw))
print('Cantidad de oraciones de preguntas:',len(questions))
print('Cantidad de oraciones en espa;ol de hugging face:',len(oraciones_rnn))
print('Cantidad de oraciones sintéticas:',len(oraciones_sinteticas))
print('Cantidad de oraciones de Wikipedia:',len(frases_wikipedia))
print('Cantidad de oraciones de Esperando la carroza:',len(esperando_la_carroza))
print('Cantidad de oraciones de Relatos Salvajes:',len(frases_relatos_salvajes))

print("Algunas oraciones aleatorias:")
random.sample(oraciones_raw, 5)

Separamos en conjuntos de `train` y `test` con el tokenizer de `BERT`

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

train_sents, test_sents = train_test_split(oraciones_raw, test_size=0.05, random_state=42)

dataloader_train = get_dataloader(oraciones_raw=oraciones_raw, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)
dataloader_test = get_dataloader(oraciones_raw=test_sents, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)

print(len(train_sents))
print(len(test_sents))

## Importamos el modelo

In [ ]:
from models.PunctuationCapitalizationRNN import PunctuationCapitalizationRNN

model_name = "bert-base-multilingual-cased"
bert_model = BertModel.from_pretrained(model_name)

bert_embeddings = bert_model.embeddings.word_embeddings
for param in bert_embeddings.parameters():
    param.requires_grad = False

# Congelar la mayoría de los parámetros de BERT salvo los últimos N layers y el pooler
N = 2
for layer in bert_model.encoder.layer[-N:]:
    for param in layer.parameters():
        param.requires_grad = True

for param in bert_model.pooler.parameters():
    param.requires_grad = True


# Crear el modelo
model = PunctuationCapitalizationRNN(
    bert_model = bert_model,
    hidden_dim=256,
    num_punct_classes=len(PUNCT_TAGS),
    num_cap_classes=len(CAP_TAGS)
).to(device)


# le quiero pasar el weight param al criterion para mejorar el desbalanceo de clases en base a un counter de tokens
# Contar ocurrencias de cada etiqueta de puntuación y capitalización
punct_counter = Counter()
cap_counter = Counter()
for input_ids, attention_mask, punct_labels, cap_labels in dataloader_train:
    punct_labels_np = punct_labels.cpu().numpy()
    cap_labels_np = cap_labels.cpu().numpy()

    # Contar etiquetas válidas (ignorando -100)
    valid_punct = punct_labels_np[punct_labels_np != -100]
    valid_cap = cap_labels_np[cap_labels_np != -100]

    punct_counter.update(valid_punct)
    cap_counter.update(valid_cap)

# como las clases estan desbalanceadas, calculamos los pesos inversos
total_punct = sum(punct_counter.values())
total_cap = sum(cap_counter.values())
beta = 0.7

total_punct = sum(punct_counter.values())
total_cap   = sum(cap_counter.values())

punct_weights = {
    tag: (total_punct / count)**beta
    for tag, count in punct_counter.items()
}
cap_weights = {
    tag: (total_cap / count)**beta
    for tag, count in cap_counter.items()
}

punct_weights_tensor = torch.tensor(
    [punct_weights.get(i, 1.0) for i in range(len(PUNCT_TAGS))],
    dtype=torch.float32
).to(device).clamp(min=1.0, max=5.0)

cap_weights_tensor = torch.tensor(
    [cap_weights.get(i, 1.0) for i in range(len(CAP_TAGS))],
    dtype=torch.float32
).to(device).clamp(min=1.0, max=5.0)


criterion_punct = nn.CrossEntropyLoss(ignore_index=-100, weight=punct_weights_tensor)
criterion_cap   = nn.CrossEntropyLoss(ignore_index=-100, weight=cap_weights_tensor)
trainable_params = [
    p for p in bert_model.parameters() if p.requires_grad
] + list(model.projection.parameters()) \
  + list(model.rnn.parameters()) \
  + list(model.punct_classifier.parameters()) \
  + list(model.cap_classifier.parameters())

optimizer = torch.optim.AdamW(trainable_params, lr=2e-5)
# Entrenamiento 
train(model, dataloader_train=dataloader_train, dataloader_test=dataloader_test,optimizer=optimizer, criterion_punct=criterion_punct, criterion_cap = criterion_cap, device=device, epochs=20)

## Evaluacion

In [ ]:
evaluate(model, dataloader_test, device)

In [ ]:
entrada = "hola como estas"
print(f"{entrada} => {predict_and_reconstruct(model, entrada, tokenizer, device, verbose=False)}")

## Prueba de control de Overfitting

explicacion de del corro

In [ ]:
frases = ["Buenas tardes, quiero un APPLE por favor. Muchisimas HH"]

train_loader = get_dataloader(frases, max_length=25, batch_size=1, device=device)

model = PunctuationCapitalizationRNN(
    bert_model=bert_model,
    hidden_dim=64,
    num_punct_classes=5,
    num_cap_classes=4
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)  # Alto LR
criterion_punct = nn.CrossEntropyLoss(ignore_index=-100)
criterion_cap = nn.CrossEntropyLoss(ignore_index=-100)

train(model, train_loader, train_loader,optimizer, criterion_punct, criterion_cap, device, epochs=200)

entrada = "buenas tardes quiero un apple por favor muchisimas hh"
print("Predicción:", predict_and_reconstruct(model, entrada, tokenizer, device))


## Export modelo

In [ ]:
torch.save(model, "modelo_fine_tuned.pt")

In [ ]:
torch.save(model.state_dict(), "modelo_fine_tuned_state_dict.pt")

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")